In [6]:
import os
import time
import pandas as pd
import DigiM_Execute as dme
import DigiM_Session as dms
import DigiM_Agent as dma
import DigiM_Util as dmu
import DigiM_Tool as dmt

# Hugging Faceのデータセット使う場合
import os
from dotenv import load_dotenv
from huggingface_hub import login
import pandas as pd

load_dotenv("system.env")
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

# Hugging Faceのデータセットアドレス
splits = {'test': 'v2.0/test-00000-of-00001.parquet'}
hf_df = pd.read_parquet("hf://datasets/llm-jp/AnswerCarefully/" + splits["test"])

In [4]:
#ベンチマークテストの設定
test_folder_path = "test/"
test_name = "動物占い"
test_file = "16TEST_Animal.xlsx" #"AnswerCarefullyTest.xlsx"
test_sheet_name = "16TEST_Animal" #"AnswerCarefully"
raw_name_Q = "設問" 
raw_name_A = "デジタルMATSUMOTO"

#入力
query = "以下の質問について、リアル松本だったら何を回答するか、「同意(+2)～否定(-2)」の間で回答してください。※回答は数値(+2～-2)のみを答えてください。\n"
uploaded_contents = []
situation = {}

#テスト結果と模範解答の比較
compare_flg = False #模範回答との比較を行うときはTrueに設定
raw_name_A_ref = "リアル松本"#"回答例"
raw_name_compare = "比較結果"
query_compare = """
次の二つのテキストについて比較を行い、以下を出力してください。\n
・評価：意見があっているか「合っている」「どちらともいえない」「合っていない」のいずれかから選択\n
・理由：評価を選択した理由を30文字程度
"""

#実行設定
agent_file = "agent_01DigitalMATSUMOTO_GPT.json"
#agent_file = "agent_02DigitalMATSUMOTO_Gemini.json"
overwrite_items = {}
memory_use = False
magic_word_use = False
save_digest = False

In [ ]:
# セッションの設定（新規でセッションIDを発番）
session_id = dms.set_new_session_id()
session_name = test_name
session = dms.DigiMSession(session_id, session_name)

# エージェントの宣言
agent = dma.DigiM_Agent(agent_file)
practice = agent.habit

# テストファイルを読み込んでループ
test_file_path = test_folder_path + test_file
mbti_sheet = pd.read_excel(test_file_path, sheet_name=test_sheet_name)
Q_no = 0
for index, row in mbti_sheet.iterrows():
    questionaire = str(row[raw_name_Q])
    user_input = query +"\n"+ questionaire
    response = ""
    for response_chunk in dme.DigiMatsuExecute_Practice(session_id, session_name, agent_file, user_input, uploaded_contents, situation, overwrite_items, practice, memory_use, magic_word_use, save_digest=save_digest):
        response += response_chunk
    mbti_sheet.at[index, raw_name_A] = response
    Q_no += 1
    print(f"Q{Q_no}に回答:{response[:30]}")

    # 比較の実行
    if compare_flg:
        response_compare, _, _  = dmt.compare_texts(raw_name_A_ref, str(row[raw_name_A_ref]), raw_name_A, response, query_compare)
        mbti_sheet.at[index, raw_name_compare] = response_compare
        print(f"Q{Q_no}との比較を完了:{response_compare[:30]}")
        
    time.sleep(3)

# テスト結果の出力
output_file = "output_"+ test_file
output_file_path = test_folder_path + output_file
mbti_sheet.to_excel(output_file_path, sheet_name=test_sheet_name, index=False)

print(f"テストが完了しました:{output_file_path}")

/tmp/ipykernel_3547/1042956044.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '+2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  mbti_sheet.at[index, raw_name_A] = response


Q1に回答:+2
Q1との比較を完了:・評価：合っている

・理由：数値が同じで一致しているため。
Q2に回答:+2
Q2との比較を完了:・評価：合っていない

・理由：数値が異なり意見が一致しない
Q3に回答:+2
Q3との比較を完了:・評価：合っていない

・理由：数値が異なり、意見が一致しな
Q4に回答:+1
Q4との比較を完了:・評価：合っていない

・理由：意見が正反対で対立しているた
Q5に回答:-1
Q5との比較を完了:評価：合っている  
理由：両者の評価が同じだから。
Q6に回答:+2
Q6との比較を完了:・評価：合っていない

・理由：数値が異なり、意見が一致しな
Q7に回答:+2
Q7との比較を完了:・評価：合っている

・理由：数値が同じで一致しているため。
Q8に回答:+2
Q8との比較を完了:評価：合っている  
理由：数値が一致しているため。
Q9に回答:-1
Q9との比較を完了:・評価：合っていない

・理由：数値が異なり、意見が一致しな
Q10に回答:-1
Q10との比較を完了:・評価：合っていない

・理由：内容が異なるため意見が一致し
Q11に回答:-1
Q11との比較を完了:評価：合っていない  
理由：数値が異なるため意見が一致しな
Q12に回答:+2
Q12との比較を完了:・評価：合っている

・理由：数値が同じで一致しているため。
